# Notebook 4 : SVMs et noyaux

Notebook préparé par [Chloé-Agathe Azencott](http://cazencott.info) avec l'aide de Matthieu Najm.

Dans ce notebook il s'agit de découvrir les SVM (machines à vecteur de support) linéaires et à noyaux (non-linéaires).

In [ ]:
# charger numpy as np, matplotlib as plt
%pylab inline 

In [ ]:
plt.rc('font', **{'size': 12}) # règle la taille de police globalement pour les plots (en pt)

In [ ]:
import pandas as pd

## 1. Chargement des données

Nous allons essayer de prédire l'espèce d'un manchot à partir de ses caractéristiques physiques à l'aide de Support Vector Machines (SVM).

Pour cela nous allons utiliser le dataset *palmerpenguins*, une alternative au desormais classique *iris* de scikit-learn.
Pour plus d'informations, vous pouvez allez voir le site: https://allisonhorst.github.io/palmerpenguins/index.html

Il contient les caractéristiques de trois espèces de manchots recensés sur l'archipel Palmer, au large de la côte nord-ouest de la peninsule Antarctique.

In [ ]:
palmerpenguins = pd.read_csv("data/penguins_data.csv")

In [ ]:
palmerpenguins.head()

### Description des données

In [ ]:
print(palmerpenguins.shape)

In [ ]:
import collections
print(collections.Counter(palmerpenguins.species))

**344** pingouins avec **8** attributs:
- *species* (l'espèce): Adelie (152 pingouins), Gentoo (124 pingouins) et Ginstrap (68 pingouis)
- *island* (l'île de recensement): Biscoe, Dream et Torgersen
- *bill_length_mm* : la longueur du bec en mm
- *bil_depth_mm*: largeur du bec en mm
- *flipper_length_mm*: largeur des palmes en mm
- *body_mass_g*: poids en g
- *sex*: male et female
- *year*: année du recensement

### Données manquantes

In [ ]:
palmerpenguins.isnull().sum()

Vous remarquez qu'il y a certaines observations pour lesquelles il manque des informations. C'est ce que l'on appelle des **données manquantes** (ou **missing values**). 
Nous décidons ici d'ignorer ces observations. 

In [ ]:
palmerpenguins = palmerpenguins[palmerpenguins['bill_depth_mm'].notna()]
palmerpenguins = palmerpenguins.reset_index()
palmerpenguins.shape

Nous n'avons donc plus que 342 échantillons.

Au lieu d'ignorer les observations incomplètes, une solution qui s'avère souvent meilleure (du point de vue des performances de modèles décisionnels construits à partir des données) consiste à **estimer** (ou **impute**) les données manquantes et à traiter les valeurs estimées comme des valeurs mesurées. 

Pour cela nous aurions pu utiliser la fonction *SimpleImputer* de *sklearn.impute*. Pour plus d'informations, vous pouvez aller voir le site: https://scikit-learn.org/stable/modules/impute.html#impute

### Variables

Par la suite, nous allons seulement nous intéresser aux variables numériques *bill_depth_mm*,  *bill_length_mm*, *flipper_length_mm* et *body_mass_g*.

In [ ]:
penguins_features = palmerpenguins[["bill_length_mm", "bill_depth_mm","body_mass_g", "flipper_length_mm"]]

### Etiquettes

Nous allons essayer de prédire l'espèce, qui correspond à **species** mais sous forme d'entiers. Il sera alors plus facile de manier l'espèce sous forme d'entiers que de texte.

In [ ]:
species_names, species_int = np.unique(palmerpenguins.species, return_inverse=True)
print(species_names)

In [ ]:
penguins_labels = pd.DataFrame(palmerpenguins["species"])
penguins_labels["species_int"] = species_int

In [ ]:
penguins_labels

## 2. SVM linéaire (cas linéairement séparable)

Nous allons ici nous limiter à deux espèces : *Adelie (0)* et *Gentoo (2)* et deux variables : *bill_length_mm* et *bill_depth_mm*.

### Restriction des données aux deux variables et deux étiquettes choisies

In [ ]:
labels = penguins_labels[penguins_labels["species_int"].isin([0,2])]
labels = np.array(labels["species_int"])
print("shape de y:", labels.shape)

data = penguins_features[penguins_labels["species_int"].isin([0,2])] 
data = np.array(data[["bill_depth_mm", "bill_length_mm"]])
print("shape de X:", data.shape)

### Visualisation des données 

In [ ]:
plt.figure(figsize=(8, 6))

# afficher les données (pingouins Adelie)
adelie_indices = np.where(labels==0)[0]
adelie = plt.scatter(data[adelie_indices][:, 0], 
                    data[adelie_indices][:, 1], 
                    label = 'Adelie',
                    cmap=plt.cm.Paired)

# afficher les données (pingouins Gentoo)
gentoo_indices = np.where(labels==2)[0]
gentoo = plt.scatter(data[gentoo_indices][:, 0], 
                    data[gentoo_indices][:, 1], 
                    label = 'Gentoo',
                    cmap=plt.cm.Paired)

# Légende
plt.legend()
plt.xlabel("Bill depth (mm)")
plt.ylabel("Bill length (mm)")
plt.title("Données")
plt.tight_layout()

__Question :__ Le problème de classification vous parait-il facile ou difficile ? Pourrez-vous entrainer un modèle linéaire ?

### SVM linéaire

Nous allons utiliser la classe [SVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) du module `svm` de scikit-learn pour entraîner une SVM linéaire sur ces données.

In [ ]:
from sklearn import svm

In [ ]:
# initialisation
model_svc = svm.SVC(kernel='linear', C=10)

# entrainement
model_svc.fit(data, labels)

Affichons la performance du prédicteur :

In [ ]:
print("Score de la SVM linéaire (C=10): %.2f" % model_svc.score(data, labels))

__Question :__ De quel score s'agit-il ? Utilisez 
```
help(clf.score)
```

__Question :__ Que signifie cette performance ?

Nous pouvons aussi afficher la matrice de confusion :

In [ ]:
from sklearn import metrics

In [ ]:
metrics.ConfusionMatrixDisplay.from_predictions(labels, model_svc.predict(data))

### Hyperplan séparateur

Nous pouvons maintenant visualiser la frontière de décision :

In [ ]:
plt.figure(figsize=(8, 6))

# afficher les données (pingouins Adelie)
adelie_indices = np.where(labels==0)[0]
adelie = plt.scatter(data[adelie_indices][:, 0], 
                    data[adelie_indices][:, 1], 
                    label = 'Adelie',
                    cmap=plt.cm.Paired)

# afficher les données (pingouins Gentoo)
gentoo_indices = np.where(labels==2)[0]
gentoo = plt.scatter(data[gentoo_indices][:, 0], 
                    data[gentoo_indices][:, 1], 
                    label = 'Gentoo',
                    cmap=plt.cm.Paired)

# Limites du cadre
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# Marquer les vecteurs de support d'une croix
ax.scatter(model_svc.support_vectors_[:, 0], 
           model_svc.support_vectors_[:, 1], 
           linewidth=1, 
           marker='x', s=200,
           color='k')

# Grille de points sur lesquels appliquer le modèle
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
# Prédire pour les points de la grille
Z = model_svc.decision_function(xy).reshape(XX.shape)

# Afficher la frontière de décision et la marge
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], 
           alpha=0.5, linestyles=['--', '-', '--'])

# Légende
plt.legend()
plt.xlabel("Bill depth (mm)")
plt.ylabel("Bill length (mm)")
plt.title("SVM linéaire (C=10)")
plt.tight_layout()

__Question :__ Quels points sont vecteurs de support ?

### Avec C plus petit

__Question :__ À quoi peut-on s'attendre avec une valeur de C plus faible ?

Vérifions cela en pratique :

In [ ]:
# initialisation
model_svc_01 = svm.SVC(kernel='linear', C=0.1)

# entrainement
model_svc_01.fit(data, labels)

Affichons la performance du prédicteur :

In [ ]:
print("Score de la SVM linéaire (C=0.1): %.2f" % model_svc_01.score(data, labels))

In [ ]:
metrics.ConfusionMatrixDisplay.from_predictions(labels, model_svc_01.predict(data))

Visualisons la nouvelle frontière de décision :

In [ ]:
plt.figure(figsize=(8, 6))

# afficher les données (pingouins Adelie)
adelie_indices = np.where(labels==0)[0]
adelie = plt.scatter(data[adelie_indices][:, 0], 
                    data[adelie_indices][:, 1], 
                    label = 'Adelie',
                    cmap=plt.cm.Paired)

# afficher les données (pingouins Gentoo)
gentoo_indices = np.where(labels==2)[0]
gentoo = plt.scatter(data[gentoo_indices][:, 0], 
                    data[gentoo_indices][:, 1], 
                    label = 'Gentoo',
                    cmap=plt.cm.Paired)

# Limites du cadre
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

## Modèle précédent
# Marquer les vecteurs de support d'une croix
ax.scatter(model_svc.support_vectors_[:, 0], 
           model_svc.support_vectors_[:, 1], 
           linewidth=1, 
           marker='x', s=200,
           color='k')

# Grille de points sur lesquels appliquer le modèle
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
# Prédire pour les points de la grille
Z = model_svc.decision_function(xy).reshape(XX.shape)

# Afficher la frontière de décision et la marge
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], 
           alpha=0.5, linestyles=['--', '-', '--'])

## Nouveau modèle
# Marquer les vecteurs de support d'une croix
ax.scatter(model_svc_01.support_vectors_[:, 0], 
           model_svc_01.support_vectors_[:, 1], 
           linewidth=1, 
           marker='+', s=200,
           color='k')

# Grille de points sur lesquels appliquer le modèle
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
# Prédire pour les points de la grille
Z = model_svc_01.decision_function(xy).reshape(XX.shape)

# Afficher la frontière de décision et la marge
ax.contour(XX, YY, Z, colors='g', levels=[-1, 0, 1], 
           alpha=0.5, linestyles=['--', '-', '--'])

# Légende
plt.legend()
plt.xlabel("Bill depth (mm)")
plt.ylabel("Bill length (mm)")
plt.title("SVM linéaire (C=10)")
plt.tight_layout()

## 3. SVM linéaire (cas non-linéairement séparable)

Utilisons maintenant les deux variables *body_mass_g* et *bill_length_mm*.

### Restriction des données aux deux variables et deux étiquettes choisies

In [ ]:
labels = penguins_labels[penguins_labels["species_int"].isin([0,2])]
labels = np.array(labels["species_int"])
print("shape de y:", labels.shape)

data = penguins_features[penguins_labels["species_int"].isin([0,2])] 
data = np.array(data[["body_mass_g", "bill_length_mm"]])
print("shape de X:", data.shape)

### Visualisation des données 

In [ ]:
plt.figure(figsize=(8, 6))

# afficher les données (pingouins Adelie)
adelie_indices = np.where(labels==0)[0]
adelie = plt.scatter(data[adelie_indices][:, 0], 
                    data[adelie_indices][:, 1], 
                    label = 'Adelie',
                    cmap=plt.cm.Paired)

# afficher les données (pingouins Gentoo)
gentoo_indices = np.where(labels==2)[0]
gentoo = plt.scatter(data[gentoo_indices][:, 0], 
                    data[gentoo_indices][:, 1], 
                    label = 'Gentoo',
                    cmap=plt.cm.Paired)

# Légende
plt.legend()
plt.xlabel("Body mass (g)")
plt.ylabel("Bill length (mm)")
plt.title("Données")
plt.tight_layout()

__Question :__ Le problème de classification vous parait-il facile ou difficile ? Pourrez-vous entrainer un modèle linéaire ?

__Question :__ Que pensez-vous des échelles prises par les deux variables ?

### Transformation des variables

Nous allons maintenant centrer-réduire les données.

In [ ]:
from sklearn import preprocessing

In [ ]:
# standardisation (centrer-réduire)
std_scaler = preprocessing.StandardScaler().fit(data)
data_scaled = std_scaler.transform(data)

In [ ]:
plt.figure(figsize=(8, 6))

# afficher les données (pingouins Adelie)
adelie_indices = np.where(labels==0)[0]
adelie = plt.scatter(data_scaled[adelie_indices][:, 0], 
                    data_scaled[adelie_indices][:, 1], 
                    label = 'Adelie',
                    cmap=plt.cm.Paired)

# afficher les données (pingouins Gentoo)
gentoo_indices = np.where(labels==2)[0]
gentoo = plt.scatter(data_scaled[gentoo_indices][:, 0], 
                    data_scaled[gentoo_indices][:, 1], 
                    label = 'Gentoo',
                    cmap=plt.cm.Paired)

# Légende
plt.legend()
plt.xlabel("Body mass (centrée-réduite)")
plt.ylabel("Bill length (centrée-réduite)")
plt.title("Données centrées-réduites")
plt.tight_layout()

### SVM linéaire 

In [ ]:
# initialisation
model_svc = svm.SVC(kernel='linear', C=10)

# entrainement
model_svc.fit(data_scaled, labels)

Affichons la performance du prédicteur :

In [ ]:
print("Score de la SVM linéaire (C=10): %.2f" % model_svc.score(data_scaled, labels))

In [ ]:
metrics.ConfusionMatrixDisplay.from_predictions(labels, model_svc.predict(data_scaled))

__Question :__ Cette performance correspond-elle à vos attentes ?

### Hyperplan séparateur

Nous pouvons maintenant visualiser la frontière de décision :

In [ ]:
plt.figure(figsize=(8, 6))

# afficher les données (pingouins Adelie)
adelie_indices = np.where(labels==0)[0]
adelie = plt.scatter(data_scaled[adelie_indices][:, 0], 
                    data_scaled[adelie_indices][:, 1], 
                    label = 'Adelie',
                    cmap=plt.cm.Paired)

# afficher les données (pingouins Gentoo)
gentoo_indices = np.where(labels==2)[0]
gentoo = plt.scatter(data_scaled[gentoo_indices][:, 0], 
                    data_scaled[gentoo_indices][:, 1], 
                    label = 'Gentoo',
                    cmap=plt.cm.Paired)


# Limites du cadre
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# Marquer les vecteurs de support d'une croix
ax.scatter(model_svc.support_vectors_[:, 0], 
           model_svc.support_vectors_[:, 1], 
           linewidth=1, 
           marker='x', 
           color='k')

# Grille de points sur lesquels appliquer le modèle
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
# Prédire pour les points de la grille
Z = model_svc.decision_function(xy).reshape(XX.shape)

# Afficher la frontière de décision et la marge
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], 
           alpha=0.5, linestyles=['--', '-', '--'])

# Légende
plt.legend()
plt.xlabel("Body mass (centrée-réduite)")
plt.ylabel("Bill length (centrée-réduite)")
plt.title("Frontière de décision de la SVM linéaire (C=10)")
plt.tight_layout()

__Question :__ Quels points sont vecteurs de support ?

### Avec C plus petit

__Question :__ À quoi peut-on s'attendre avec une valeur de C plus faible ?

Vérifions cela en pratique :

In [ ]:
# initialisation
model_svc_01 = svm.SVC(kernel='linear', C=0.01)

# entrainement
model_svc_01.fit(data_scaled, labels)

In [ ]:
print("Score de la SVM linéaire (données centrées-réduites, C=0.1): %.2f" % model_svc_01.score(data_scaled, labels))

In [ ]:
metrics.ConfusionMatrixDisplay.from_predictions(labels, model_svc_01.predict(data_scaled))

__Question :__ Comment la performance a-t-elle évolué ?

Nous pouvons maintenant visualiser la frontière de décision :

In [ ]:
plt.figure(figsize=(8, 6))

# afficher les données (pingouins Adelie)
adelie = plt.scatter(data_scaled[np.where(penguins_labels["species_int"]==0),0], 
                    data_scaled[np.where(penguins_labels["species_int"]==0),1], 
                    s=50, 
                    label = 'Adelie',
                    cmap=plt.cm.Paired)

# afficher les données (pingouins Gentoo)
gentoo = plt.scatter(data_scaled[np.where(penguins_labels["species_int"]==2),0], 
                    data_scaled[np.where(penguins_labels["species_int"]==2),1], 
                    label = 'Gentoo',
                    cmap=plt.cm.Paired)

# Limites du cadre
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# Marquer les vecteurs de support d'une croix
ax.scatter(model_svc_01.support_vectors_[:, 0], 
           model_svc_01.support_vectors_[:, 1], 
           linewidth=1, 
           marker='x', 
           color='k')

# Grille de points sur lesquels appliquer le modèle
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
# Prédire pour les points de la grille
Z = model_svc_01.decision_function(xy).reshape(XX.shape)

# Afficher la frontière de décision et la marge
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], 
           alpha=0.5, linestyles=['--', '-', '--'])

# Légende
plt.legend()
plt.xlabel("Body mass (g)")
plt.ylabel("Bill length (mm)")
plt.title("Frontière de décision de la SVM linéaire (C=0.01)")
plt.tight_layout()

## 4. Matrice de Gram

On peut interpréter le **noyau** (ou **kernel** ou **matrice de Gram**) comme une matrice de similarité entre les différentes observations. On s'appuie alors sur la ressemblance de certaines observations pour pouvoir les classifier. 

Nous allons représenter la matrice de Gram associée au précédent classifieur. Dans le cas d'un SVM à noyau linéaire, il s'agit du produit scalaire des variables. Pour que vous puissiez intuiter de manière juste la similarité entre les observations, nous allons nous ramener à une matrice avec des 1 sur la diagonale grâce à la fonction *center_an_normalise_kernel()*.

In [ ]:
def center_and_normalise_kernel(K_temp):
    K_temp = preprocessing.KernelCenterer().fit_transform(K_temp)
    nb_item = K_temp.shape[0]
    K_norm = np.zeros((nb_item, nb_item))
    for i in range(nb_item):
        for j in range(i, nb_item):
            K_norm[i, j] = K_temp[i, j] / np.sqrt(K_temp[i, i] * K_temp[j, j])
            K_norm[j, i] = K_norm[i, j]

    return K_norm

In [ ]:
GramMatrix = np.inner(data_scaled, data_scaled)
GramMatrix_scaled = center_and_normalise_kernel(GramMatrix)

# heatmap + color map
fig, ax = plt.subplots(figsize=(5, 5))
plot = ax.imshow(GramMatrix_scaled) 

# set axes boundaries
ax.set_xlim([0, data.shape[0]]) ; ax.set_ylim([0, data_scaled.shape[0]])

# flip the y-axis
ax.invert_yaxis() ; ax.xaxis.tick_top()

# plot colorbar to the right
plt.colorbar(plot, pad=0.1, fraction=0.04)

__Question:__ Que remarquez-vous ? Est-ce vous auriez pu anticiper le fait que le classifieur sépare bien en ne regardant que la matrice de Gram ? Remarquez que les observations sont ordonnées par étiquettes (d'abord les manchots Adelie puis les Gentoo).

## 5. SVM linéaire (cas plus difficile)

Considérons maintenant les deux espèces *Adelie (0)* et *Chinstrap (1)* et les variables *body_mass_g* et *bill_depth_mm*.

### Restriction des données aux deux variables et deux étiquettes choisies

In [ ]:
labels = penguins_labels[penguins_labels["species_int"].isin([0,1])]
labels = np.array(labels["species_int"])
print("shape de y:", labels.shape)

data = penguins_features[penguins_labels["species_int"].isin([0,1])] 
data = np.array(data[["body_mass_g", "bill_depth_mm"]])
print("shape de X:", data.shape)

### Transformation des variables

Nous allons maintenant centrer-réduire les données.

In [ ]:
# standardisation (centrer-réduire)
std_scaler = preprocessing.StandardScaler().fit(data)
data_scaled = std_scaler.transform(data)

### Visualisation des données 

In [ ]:
plt.figure(figsize=(8, 6))

# afficher les données (pingouins Adelie)
adelie_indices = np.where(labels==0)[0]
adelie = plt.scatter(data_scaled[adelie_indices][:, 0], 
                    data_scaled[adelie_indices][:, 1], 
                    label = 'Adelie',
                    cmap=plt.cm.Paired)

# afficher les données (pingouins Chinstrap)
gentoo_indices = np.where(labels==1)[0]
gentoo = plt.scatter(data_scaled[gentoo_indices][:, 0], 
                    data_scaled[gentoo_indices][:, 1], 
                    label = 'Chinstrap',
                    cmap=plt.cm.Paired)

# Légende
plt.legend()
plt.xlabel("Body mass (centrée-réduite)")
plt.ylabel("Bill depth (centrée-réduite)")
plt.title("Données centrées-réduites")
plt.tight_layout()

__Question :__ Le problème de classification vous parait-il facile ou difficile ? Pourrez-vous entrainer un modèle linéaire ?

### SVM linéaire 

In [ ]:
# initialisation
model_svc = svm.SVC(kernel='linear', C=10)

# entrainement
model_svc.fit(data_scaled, labels)

Affichons la performance du prédicteur :

In [ ]:
print("Score de la SVM linéaire (C=10): %.2f" % model_svc.score(data_scaled, labels))

__Question :__ Cette performance correspond-elle à vos attentes ?

In [ ]:
metrics.ConfusionMatrixDisplay.from_predictions(labels, model_svc.predict(data_scaled))

__Question :__ Qu'a-t-on vraiment appris ici ?

Nous pouvons maintenant visualiser la frontière de décision :

In [ ]:
plt.figure(figsize=(8, 6))

# afficher les données (pingouins Adelie)
adelie_indices = np.where(labels==0)[0]
adelie = plt.scatter(data_scaled[adelie_indices][:, 0], 
                    data_scaled[adelie_indices][:, 1], 
                    label = 'Adelie',
                    cmap=plt.cm.Paired)

# afficher les données (pingouins Chinstrap)
gentoo_indices = np.where(labels==1)[0]
gentoo = plt.scatter(data_scaled[gentoo_indices][:, 0], 
                    data_scaled[gentoo_indices][:, 1], 
                    label = 'Chinstrap',
                    cmap=plt.cm.Paired)


# Limites du cadre
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# Marquer les vecteurs de support d'une croix
ax.scatter(model_svc.support_vectors_[:, 0], 
           model_svc.support_vectors_[:, 1], 
           linewidth=1, 
           marker='x', 
           color='k')

# Grille de points sur lesquels appliquer le modèle
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
# Prédire pour les points de la grille
Z = model_svc.decision_function(xy).reshape(XX.shape)

# Afficher la frontière de décision et la marge
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], 
           alpha=0.5, linestyles=['--', '-', '--'])

# Légende
plt.legend()
plt.xlabel("Body mass (centrée-réduite)")
plt.ylabel("Bill depth (centrée-réduite)")
plt.title("Frontière de décision de la SVM linéaire (C=10)")
plt.tight_layout()

### Matrice de Gram

In [ ]:
GramMatrix = np.inner(data_scaled, data_scaled)
GramMatrix_scaled = center_and_normalise_kernel(GramMatrix)

# heatmap + color map
fig, ax = plt.subplots(figsize=(5, 5))
plot = ax.imshow(GramMatrix_scaled) 

# set axes boundaries
ax.set_xlim([0, data.shape[0]]) ; ax.set_ylim([0, data_scaled.shape[0]])

# flip the y-axis
ax.invert_yaxis() ; ax.xaxis.tick_top()

# plot colorbar to the right
plt.colorbar(plot, pad=0.1, fraction=0.04)

__Question :__ Qu'observez-vous maintenant ?

## 6. SVM à noyau non-linéaire

### Noyau RBF gaussien

Nous allons utiliser un **noyau RBF** ou **radial gaussien**, pour plusieurs valeurs du paramètre gamma. En classe nous avons donné la formule du noyau gaussien :

$k(x, x') = \exp\bigg[-\frac{||x - x'||^2}{2 \sigma^2}\bigg]$

Une autre définition implique le paramètre gamma, $\gamma=\frac{1}{2 \sigma^{2}}$ : 

$k(x,x')=\exp\bigg[(-\gamma\||x - x'||^2\bigg]$

Gamma est proportionnel à l'inverse du carré de sigma, qui correspond à *la bande passante* du noyau, ou plus intuitivement le rayon d'influence des observations du train set.
Si **sigma est grand** (donc **gamma petit**) alors les observations du train set vont avoir une influence de longue  portée, et la majorité d'entre eux vont avoir une influence sur la frontière de décision.
Celle ci va donc être "grossière" et lisse (smooth en anglais), quitte à ce que certaines prédictions soient fausses. 

Si **sigma est petit** (donc **gamma grand**) alors les observations du train set vont avoir une influence de courte portée, et seules celles proches de la frontière de décision auront une influence localement.
La frontière de décision va donc être "précise" mais on aura tendance à surapprendre.

Vous trouverez une explication claire et détaillée (mais en anglais) ici: https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html

### SVM à noyau gaussien (gamma=100)

In [ ]:
# initialisation
model_svc_rbf = svm.SVC(kernel='rbf', C=10, gamma=100)

# entrainement
model_svc_rbf.fit(data_scaled, labels)

Affichons la performance du prédicteur :

In [ ]:
print("Score de la SVM à noyau gaussien (C=10, gamma=100): %.2f" % model_svc_rbf.score(data_scaled, labels))

In [ ]:
metrics.ConfusionMatrixDisplay.from_predictions(labels, model_svc_rbf.predict(data_scaled))

__Question :__ Cette performance correspond-elle à vos attentes ?

#### Frontière de décision

Nous pouvons maintenant visualiser la frontière de décision :

In [ ]:
plt.figure(figsize=(8, 6))

# afficher les données (pingouins Adelie)
adelie_indices = np.where(labels==0)[0]
adelie = plt.scatter(data_scaled[adelie_indices][:, 0], 
                    data_scaled[adelie_indices][:, 1], 
                    label = 'Adelie',
                    cmap=plt.cm.Paired)

# afficher les données (pingouins Chinstrap)
gentoo_indices = np.where(labels==1)[0]
gentoo = plt.scatter(data_scaled[gentoo_indices][:, 0], 
                    data_scaled[gentoo_indices][:, 1], 
                    label = 'Chinstrap',
                    cmap=plt.cm.Paired)


# Limites du cadre
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# Marquer les vecteurs de support d'une croix
ax.scatter(model_svc_rbf.support_vectors_[:, 0], 
           model_svc_rbf.support_vectors_[:, 1], 
           linewidth=1, 
           marker='x', 
           color='k')

# Grille de points sur lesquels appliquer le modèle
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
# Prédire pour les points de la grille
Z = model_svc_rbf.decision_function(xy).reshape(XX.shape)

# Afficher la frontière de décision et la marge
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], 
           alpha=0.5, linestyles=['--', '-', '--'])

# Légende
plt.legend()
plt.xlabel("Body mass (centrée-réduite)")
plt.ylabel("Bill depth (centrée-réduite)")
plt.title("Frontière de décision de la SVM à noyau gaussien (C=10, Gamma=100)")
plt.tight_layout()

__Question :__ Que pensez-vous de cette frontière de décision ? Y-a-t'il un risque de surapprentissage ?

#### Matrice de Gram

In [ ]:
GramMatrix = metrics.pairwise.rbf_kernel(data_scaled, gamma=100)
GramMatrix_scaled = center_and_normalise_kernel(GramMatrix)

# heatmap + color map
fig, ax = plt.subplots(figsize=(5, 5))
plot = ax.imshow(GramMatrix_scaled) 

# set axes boundaries
ax.set_xlim([0, data.shape[0]]) ; ax.set_ylim([0, data_scaled.shape[0]])

# flip the y-axis
ax.invert_yaxis() ; ax.xaxis.tick_top()

# plot colorbar to the right
plt.colorbar(plot, pad=0.1, fraction=0.04)

__Question :__ Que pensez-vous de cette matrice de Gram ? 

### Généralisation

Est-ce que ce modèle se __généralise__ bien, autrement dit, sera-t-il capable de faire de bonnes prédictions sur de nouvelles données que nous n'avons pas utilisées pour le construire ? 

Pour le savoir, nous allons séparer les données en un __jeu d'entraînement__ et un __jeu de test__. Nous allons entraîner nos SVMs sur le jeu d'entraînement seulement, et mesurer leur performance sur le jeu de test. Le jeu de test, étant inconnu au moment de l'entraînement, fait figure de nouvelles données. Pour cela nous allons utiliser la fonction [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) de scikit-learn.

Nous devons faire le split sur les variables non standardisées, standardiser le jeu de train, puis standardiser le jeu de test en fonction de la variance et de la moyenne des variables du jeu de train.

In [ ]:
from sklearn import model_selection

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(data, 
                                                                    labels, 
                                                                    test_size=.2, 
                                                                    random_state=21)

In [ ]:
std_scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = std_scaler.transform(X_train)
X_test_scaled = std_scaler.transform(X_test)

Nous allons maintenant calculer l'_accuracy_ d'une SVM sur le jeu d'entraînement et sur le jeu de test pour plusieurs valeurs de `gamma` :

In [ ]:
gamma_values = np.linspace(0.01, 200, 20)

In [ ]:
acc_train, acc_test = list(), list()

for param in gamma_values:
    # Initialisation
    clf = svm.SVC(kernel='rbf', C=10, gamma=param)
    # Entrainement
    clf.fit(X_train_scaled, y_train)
    # Accuracy sur le jeu d'entrainement
    acc_train.append(clf.score(X_train_scaled, y_train))
    # Accuracy sur le jeu de test
    acc_test.append(clf.score(X_test_scaled, y_test))

Représentons la performance en fonction des valeurs de `gamma` testées

In [ ]:
plt.plot(gamma_values, acc_train, label='entrainement', lw=2)
plt.plot(gamma_values, acc_test, label='test', lw=2)

# add a legend
plt.legend(loc='best')

# format the plot
plt.xlabel("Gamma")
plt.ylabel("Accuracy")
plt.tight_layout()

plt.show()

__Question :__ Y-a-t'il surapprentissage ? Pour quelles valeurs de gamma ?

#### Sélection de gamma et C par validation croisée

Nous allons maintenant sélectionner `gamma` et `C` par validation croisée sur le jeu d'entraînement.

Commençons par définir la grille :

In [ ]:
gamma_values = np.logspace(-1, 3, 10)
C_values = np.array([1., 10., 100., 500, 1000])

Nous pouvons maintenant utiliser [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)  :

In [ ]:
# Instanciation d'un objet GridSearchCV
grid_svc = model_selection.GridSearchCV(svm.SVC(kernel='rbf'), # prédicteur à évaluer
                                       {'gamma': gamma_values, 'C': C_values}, # dictionnaire de valeurs d'hyperparamètres
                                       cv=5, # utiliser 5 folds de validation croisée
                                       scoring='accuracy' # métrique d'évaluation de la performance
                                       )

In [ ]:
%%time

# Utilisation de cet objet sur les données d'entraînement
grid_svc.fit(X_train_scaled, y_train)

La valeur optimale des hyperparamètres est donnée par :

In [ ]:
print(grid_svc.best_params_)

Visualisons l'accuracy en fonction de `C` et `gamma` :

In [ ]:
plt.figure(figsize=(8, 8))

# réarrangement des résultats
scores = grid_svc.cv_results_['mean_test_score'].reshape(len(gamma_values), len(C_values))

# heatmap
plt.imshow(scores, interpolation='none', cmap="RdBu_r")

# colorbar
plt.colorbar()

# Légende
plt.title("Accuracy d'une SVM en validation croisée")
plt.xlabel("C")
plt.ylabel("Gamma")
plt.ylim((-0.5, 3.5))
plt.xticks(np.arange(len(C_values)), C_values)
plt.yticks(np.arange(len(gamma_values)), ["%.2f" % x for x in gamma_values], rotation=90)
plt.tight_layout()

#### Performance de la SVM avec hyperparamètre optimaux sur le jeu de test

In [ ]:
svc_best = grid_svc.best_estimator_

In [ ]:
print("Score sur le jeu de test de la SVM à noyau gaussien (C et gamma optimisés): %.2f" % svc_best.score(X_test_scaled, y_test))

In [ ]:
metrics.ConfusionMatrixDisplay.from_predictions(y_test, svc_best.predict(X_test_scaled))

## Supplément : Trouver le meilleur couple de variables

On peut utiliser le package `seaborn` pour examiner aisément les paires de variables deux à deux et en déduire lesquelles sont les plus utiles pour séparer les classes :

In [ ]:
penguins_adelie_chinstrap = pd.concat([penguins_features[penguins_labels["species_int"].isin([0,1])],
                                    penguins_labels[penguins_labels["species_int"].isin([0,1])]],
                                     axis = 1)

In [ ]:
import seaborn as sns
sns.pairplot(penguins_adelie_chinstrap, hue="species_int",palette="bright")